# Usługa uczenia maszynowego w Microsoft SQL Server

Usługa uczenia maszynowego w Microsoft SQL Server to funkcja programu, która umożliwia uruchamianie skryptów w językach Python i R bezpośrednio z bazy danych. Dzięki temu, możliwe jest wykorzystanie różnych narzędzi i bibliotek do przeprowadzania analiz predykcyjnych i uczenia maszynowego na danych zgromadzonych w bazie bez konieczności przenoszenia ich poza SQL Server. Usługa ta jest przydatna do automatyzacji i usprawnienia procesów związanych z uczeniem maszynowym.

Standardowo obraz dockera Microsoft SQL Server __nie posiada__ serwisu SQL Machine Learning, aby stworzyć taki kontener trzeba posłużyć się tym przykładem [dockerfile'a](https://github.com/Microsoft/mssql-docker/tree/master/linux/preview/examples/mssql-mlservices).

# Skrypty Python w SQL Server

[**Opis języka Python**](https://docs.python.org/3/tutorial/index.html)

Python jest zainstalowany niezależnie w systemie Microsoft SQL Server w ramach SQL Machine Learning.

Skrypt w języku Python można uruchomić przez przekazanie go jako argument do procedury składowanej [sp\_execute\_external\_script](https://learn.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-execute-external-script-transact-sql?view=sql-server-ver16). Ta procedura systemowa pozwala na uruchomienie środowiska Pythona w kontekście uczenia maszynowego SQL, przesyłanie danych do Pythona, zarządzanie sesjami użytkownika Pythona i zwracanie wyników do klienta.

## Zezwolenie na działanie procedury sp\_execute\_external\_script

In [ ]:
sp_configure 'external scripts enabled', 1;
RECONFIGURE;

## Przykłady użycia skryptów Python w SQL

### Parametry procedury sp\_execute\_external\_script:

**@language** – język. Przyjmowane wartości to R, Python i język zdefiniowany za pomocą [CREATE EXTERNAL LANGUAGE](https://learn.microsoft.com/en-us/sql/t-sql/statements/create-external-language-transact-sql?view=sql-server-ver16). \
**@script** – treść skryptu. Cały skrypt musi być zapisany jako tekst Unicode. Można również przekazać tekst przez zmienną typu nvarchar.

In [61]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import sys
print("Python version:", sys.version)
'

STDOUT message(s) from external script: 
Python version: 3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]

Total execution time: 00:00:02.798

In [62]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
ls = [1, 2.0, 3, "abc"]

for element in ls:
    print(element)
'

STDOUT message(s) from external script: 
1
2.0
3
abc

Total execution time: 00:00:02.723

Uczenie maszynowe w systemie Microsoft SQL Server w zakresie obsługi języka Python opiera się na pakiecie pandas. Podstawowy obiekt, poprzez który przekazywane są dane do i z Pythona to ramka danych (DataFrame), dlatego dane wejściowe muszą być tabelaryczne, np. utworzone zdaniem SELECT. Dane wyjściowe również muszą być przedstawione w postaci tego typu obiektu.

**@input\_data\_1 = N'input\_data\_1'** - Dane wejściowe wykorzystywane przez zewnętrzny skrypt w postaci zapytania Transact-SQL.  
**@input\_data\_1\_name = N'input\_data\_1\_name'** - Nazwa zmiennej używanej do reprezentowania zapytania zdefiniowanego przez @input\_data\_1. Domyślną wartością jest **InputDataSet**.

**@output\_data\_1\_name = N'output\_data\_1\_name'** - Nazwa zmiennej w zewnętrznym skrypcie, która zawiera dane zwracane do serwera SQL po zakończeniu wywołania procedury składowanej. Domyślną wartością jest **OutputDataSet**.

**Uwaga** Notebook wyświetla stdout z procedury w jednej linii, dla bardziej skomplikowanego wyjścia poleca się użycie zwykłego querry.

In [63]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT 1, N''ABC'''

(1 row affected)

Total execution time: 00:00:02.839

(No column name),(No column name)
1,ABC


In [64]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import pandas as pd
import matplotlib.pyplot as plt

if isinstance(InputDataSet, pd.DataFrame):
    print("InputDataSet is a DataFrame")
'
    , @input_data_1 = N'SELECT 1, N''ABC'''

STDOUT message(s) from external script: 
InputDataSet is a DataFrame

Total execution time: 00:00:03.878

**WITH RESULTS SETS** - klauzula pozwala na określenie schematu tabeli, która jest zwracana przez środowisko SQL. Ten schemat musi pasować do ramki danych, która została przesłana z środowiska wykonawczego i zawierać typy danych z języka SQL.

In [65]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT 1, N''ABC'''
WITH RESULT SETS(([Number] INT, Text NVARCHAR(10)));
GO

(1 row affected)

Total execution time: 00:00:02.915

Number,Text
1,ABC


In [66]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'SQL_out = SQL_in'
    , @input_data_1 = N'SELECT TOP 5 ProductID, ProductName, CategoryID FROM Northwind.dbo.Products'
    , @input_data_1_name = N'SQL_in'
    , @output_data_1_name = N'SQL_out'
WITH RESULT SETS((ProductID INT, ProductName NVARCHAR(40), CategoryID INT));
GO

(5 rows affected)

Total execution time: 00:00:02.854

ProductID,ProductName,CategoryID
1,Chai,1
2,Chang,1
3,Aniseed Syrup,2
4,Chef Anton's Cajun Seasoning,2
5,Chef Anton's Gumbo Mix,2


Pewne typy danych nie są wspierane, należy wówczas wykorzystać funkcję CAST, warto też zamieniać na właściwsze typy w wynikach za pomocą WITH RESULTS SETS.

In [67]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT ProductID, ProductName, UnitPrice FROM Northwind.dbo.Products;'

: Msg 39004, Level 16, State 20, Line 1
A 'Python' script error occurred during execution of 'sp_execute_external_script' with HRESULT 0x80004004.

: Msg 39019, Level 16, State 2, Line 1
An external script error occurred: 

Unsupported input data type in column 'UnitPrice'.  Supported types: bit, tinyint, smallint, int, bigint, uniqueidentifier, real, float, char, varchar, nchar, nvarchar, varbinary, date, datetime, smalldatetime.
SqlSatelliteCall error: Unsupported input data type in column 'UnitPrice'.  Supported types: bit, tinyint, smallint, int, bigint, uniqueidentifier, real, float, char, varchar, nchar, nvarchar, varbinary, date, datetime, smalldatetime.

STDOUT message(s) from external script: 
SqlSatelliteCall function failed. Please see the console output for more information.
Traceback (most recent call last):
 File "/opt/mssql/mlservices/libraries/PythonServer/revoscalepy/computecontext/RxInSqlServer.py", line 605, in rx_sql_satellite_call
 rx_native_call("SqlSatelliteCall", params)
 File "/opt/mssql/mlservices/libraries/PythonServer/revoscalepy/RxSerializable.py", line 375, in rx_native_call
 ret = px_call(functionname, params)
RuntimeError: revoscalepy function failed.

Total execution time: 00:00:02.672

In [68]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'OutputDataSet = InputDataSet'
    , @input_data_1 = N'SELECT TOP 5 ProductID, ProductName, CAST(UnitPrice AS FLOAT) 
	FROM Northwind.dbo.Products; '
WITH RESULT SETS((ProductID INT, ProductName NVARCHAR(10), UnitPrice MONEY));
GO

(5 rows affected)

Total execution time: 00:00:02.736

ProductID,ProductName,UnitPrice
1,Chai,"18,00"
2,Chang,"19,00"
3,Aniseed Sy,"10,00"
4,Chef Anton,"22,00"
5,Chef Anton,"21,35"


Chcąc dodać dodatkowe parametry do skryptu zewnętrznego, należy określić ich nazwy i typy danych w parametrze **@params**.

In [69]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'print(a, b)'
    , @params = N' @a int, @b float'
    , @a = 5
    , @b = 2.0

STDOUT message(s) from external script: 
5 2.0

Total execution time: 00:00:02.788

In [ ]:
DROP PROCEDURE IF EXISTS ParamsExample
GO

CREATE PROCEDURE ParamsExample (
      @param1 INT
    , @param2 FLOAT
    )
AS
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'print(a, b)'
    , @params = N' @a int, @b float'
    , @a = @param1
    , @b = @param2
GO

In [73]:
EXEC ParamsExample 100, 50

STDOUT message(s) from external script: 
100 50.0

Total execution time: 00:00:02.810

Procedura może przyjmować tylko jeden zestaw danych wejściowych i zwracać tylko jeden zestaw danych wyjściowych. Jednakże, w kodzie Pythona, istnieje możliwość odwoływania się do innych zestawów danych. Dodatkowo, zestaw danych nie jest jedynym typem danych, które można zwrócić. W celu zwrócenia wartości z dowolnego parametru, można dodać słowo kluczowe OUTPUT. \
Wiecej informacji o parametrach można znaleźć w dokumnetacji [sp_execute_external_script](https://learn.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-execute-external-script-transact-sql?view=sql-server-ver15#arguments).

## Standardowe biblioteki Python w SQL Machine Learning
Listę zainstalowanych pakietów Pythona, w tym wersję, można wyświetlić przez poniższy skrypt:

In [ ]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import pkg_resources
import pandas as pd

dists = [str(d) for d in pkg_resources.working_set]
OutputDataSet = pd.DataFrame(dists)
'
WITH RESULT SETS(([Package] NVARCHAR(max)))
GO

### Pandas & Numpy
SQL Machine Learning, korzystające z języka Python, opiera się na pakiecie pandas, który jest bardzo przydatny do pracy z danymi tabelarycznymi. Głównym narzędziem przekazywania danych do i z Pythona jest ramka danych (DataFrame), która odpowiada tabeli z wieloma kolumnami. Pojedyncza kolumna ramki danych jest podobna do jednowymiarowej tablicy lub słownika, i nazywana jest serią (Series). W ramce danych pojedyncza wartość jest znana jako komórka (Cell) i dostępna jest przez indeks.

Numpy jest głównie wykorzystywane do obliczeń matematycznych z wieloma funkcjami statystycznymi.

### Przykłady z pandas Series
Najlepiej kwerendy uruchomić we zwykłym querry.

In [75]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import pandas as pd

a = 1
b = 2
c = a/b
d = a*b

s = pd.Series([c,d,(2,3),"abc"])
print(s)
'

STDOUT message(s) from external script: 
0 0.5
1 2
2 (2, 3)
3 abc
dtype: object

Total execution time: 00:00:02.830

In [76]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
a = 1
b = 2
c = a/b
s = pandas.Series(c, index =["simple math example 1", "simple math example 2", "simple math example 3"])
print(s)
'

STDOUT message(s) from external script: 
simple math example 1 0.5
simple math example 2 0.5
simple math example 3 0.5
dtype: float64

Total execution time: 00:00:02.759

[Dokumentacja pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)

### Liczby pseudolosowe w numpy
Losowe próbki z rozkładu normalnego [numpy.random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)

In [ ]:
CREATE OR ALTER PROCEDURE MyPyNorm (
      @size INT
    , @mean INT
    , @scale INT
    )
AS
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import numpy
import pandas
OutputDataSet = pandas.DataFrame(numpy.random.normal(size=size, loc=mean, scale=scale));
'
    , @params = N' @size int, @mean int, @scale int'
    , @size = @size
    , @mean = @mean
    , @scale = @scale
WITH RESULT SETS(([Density] FLOAT NOT NULL));
GO

In [78]:
EXEC MyPyNorm 5, 50, 10

(5 rows affected)

Total execution time: 00:00:02.817

Density
"64,44368652331016"
"50,35221807398348"
"41,30177472001347"
"51,84385652810873"
"47,69796004852747"


Losowe próbki z liczb całkowitych [numpy.random.randint](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)

In [ ]:
CREATE OR ALTER PROCEDURE MyPyRandInt (
        @size INT,
        @low INT,
	    @high INT
    )
AS
EXECUTE sp_execute_external_script @language = N'Python'
    , @script = N'
import pandas as pd
import numpy as np
values = np.random.randint(low=low, high=high, size=size)
OutputDataSet = pd.DataFrame(values);
'
    , @params = N' @low int, @high int, @size int'
    , @low = @low
    , @high = @high
    , @size = @size
WITH RESULT SETS(([Number] INT NOT NULL));
GO

In [80]:
EXEC MyPyRandInt 5, 50, 100

(5 rows affected)

Total execution time: 00:00:02.696

Number
52
83
70
52
90


## Przykład użycia uczenia maszynowego w SQL Machine Learning

Użyjemy model [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) do klasyfikacji win na podstawie analizy chemicznej w trzech rodzajach win.


In [ ]:
CREATE DATABASE winesql
GO


In [ ]:
USE winesql
GO

**1. Przygotowanie Tabeli przechowującej dane do trenowania**

In [ ]:
DROP TABLE IF EXISTS wine_data;
GO

CREATE TABLE wine_data (
    id INT NOT NULL IDENTITY PRIMARY KEY,
    Alcohol FLOAT NOT NULL,
    MalicAcid FLOAT NOT NULL,
    Ash FLOAT NOT NULL,
    AlcalinityOfAsh FLOAT NOT NULL,
    Magnesium FLOAT NOT NULL,
    TotalPhenols FLOAT NOT NULL,
    Flavanoids FLOAT NOT NULL,
    NonflavanoidPhenols FLOAT NOT NULL,
    Proanthocyanins FLOAT NOT NULL,
    ColorPntensity FLOAT NOT NULL,
    Hue FLOAT NOT NULL,
    ProteinConcentration FLOAT NOT NULL,
    Proline FLOAT NOT NULL,
    WineId INT NOT NULL
);

**2. Tabela przechowująca wytrenowany model (modele) – sam model musi być zserializowany i przechowywany w kolumnie typu varbinary(max)**

In [ ]:
DROP TABLE IF EXISTS wine_models;
GO

CREATE TABLE wine_models (
  model_name VARCHAR(50) NOT NULL DEFAULT('default model') PRIMARY KEY,
  model VARBINARY(MAX) NOT NULL
);
GO

**3. Wypełnienie tabeli wine_data danymi**

In [ ]:
CREATE OR ALTER PROCEDURE get_wine_dataset
AS
BEGIN
EXEC sp_execute_external_script @language = N'Python', 
@script = N'
from sklearn import datasets
import pandas as pd

wine = datasets.load_wine()
wine_data = pd.DataFrame(wine.data)
wine_data["WineId"] = wine.target
', 
@output_data_1_name = N'wine_data'
WITH RESULT SETS ((
    Alcohol FLOAT NOT NULL,
    MalicAcid FLOAT NOT NULL,
    Ash FLOAT NOT NULL,
    AlcalinityOfAsh FLOAT NOT NULL,
    Magnesium FLOAT NOT NULL,
    TotalPhenols FLOAT NOT NULL,
    Flavanoids FLOAT NOT NULL,
    NonflavanoidPhenols FLOAT NOT NULL,
    Proanthocyanins FLOAT NOT NULL,
    ColorPntensity FLOAT NOT NULL,
    Hue FLOAT NOT NULL,
    ProteinConcentration FLOAT NOT NULL,
    Proline FLOAT NOT NULL,
    WineId INT NOT NULL
    ));
END;
GO

In [ ]:
INSERT INTO wine_data
EXEC dbo.get_wine_dataset;

In [86]:
SELECT TOP 5 * FROM wine_data

(5 rows affected)

Total execution time: 00:00:00.008

id,Alcohol,MalicAcid,Ash,AlcalinityOfAsh,Magnesium,TotalPhenols,Flavanoids,NonflavanoidPhenols,Proanthocyanins,ColorPntensity,Hue,ProteinConcentration,Proline,WineId
1,"14,23","1,71","2,43","15,6",127,"2,8","3,06","0,28","2,29","5,64","1,04","3,92",1065,0
2,"13,2","1,78","2,14","11,2",100,"2,65","2,76","0,26","1,28","4,38","1,05","3,4",1050,0
3,"13,16","2,36","2,67","18,6",101,"2,8","3,24","0,3","2,81","5,68","1,03","3,17",1185,0
4,"14,37","1,95","2,5","16,8",113,"3,85","3,49","0,24","2,18","7,8","0,86","3,45",1480,0
5,"13,24","2,59","2,87",21,118,"2,8","2,69","0,39","1,82","4,32","1,04","2,93",735,0


**4. Utworzenie modelu i wytrenowanie modelu**

In [ ]:
CREATE OR ALTER PROCEDURE generate_wine_model (
        @estimators INT,
        @depth INT,
        @trained_model VARBINARY(max) OUTPUT
)
AS
BEGIN
EXEC sp_execute_external_script @language = N'Python'
        , @script = N'
import pickle
from sklearn.ensemble import RandomForestClassifier

# model
print("RandomForest estinamtors:", estimators, "depth:", depth)
random_forest = RandomForestClassifier(n_estimators=estimators, max_depth=depth)

# target
target = wine_data["WineId"]

wine_data.drop(labels=["WineId", "id"], axis=1, inplace=True)

random_forest.fit(wine_data, target)

trained_model = pickle.dumps(random_forest)
'
        , @input_data_1 = N'SELECT * from wine_data'
        , @input_data_1_name = N'wine_data'
        , @params = N'@estimators INT, @depth INT, @trained_model varbinary(max) OUTPUT'
        , @estimators = @estimators
        , @depth = @depth
        , @trained_model = @trained_model OUTPUT;
END;
GO

In [88]:
DECLARE @model VARBINARY(max);

EXEC generate_wine_model @estimators=2, @depth=2, @trained_model=@model OUTPUT;

INSERT INTO wine_models VALUES (N'RandomForest', @model)

STDOUT message(s) from external script: 
RandomForest estinamtors: 2 depth: 2

(1 row affected)

Total execution time: 00:00:04.281

In [89]:
SELECT * FROM wine_models

(1 row affected)

Total execution time: 00:00:00.020

model_name model RandomForest 0x800363736B6C6561726E2E656E73656D626C652E666F726573740A52616E646F6D466F72657374436C61737369666965720A7100298171017D710228580E000000626173655F657374696D61746F72710363736B6C6561726E2E747265652E747265650A4465636973696F6E54726565436C61737369666965720A7104298171057D7106285809000000637269746572696F6E7107580400000067696E697108580800000073706C69747465727109580400000062657374710A58090000006D61785F6465707468710B4E58110000006D696E5F73616D706C65735F73706C6974710C4B0258100000006D696E5F73616D706C65735F6C656166710D4B0158180000006D696E5F7765696768745F6672616374696F6E5F6C656166710E470000000000000000580C0000006D61785F6665617475726573710F4E580C00000072616E646F6D5F737461746571104E580E0000006D61785F6C6561665F6E6F64657371114E58150000006D696E5F696D7075726974795F6465637265617365711247000000000000000058120000006D696E5F696D7075726974795F73706C697471134E580C000000636C6173735F77656967687471144E5807000000707265736F727471158958100000005F736B6C6561726E5F76657273696F6E71165806000000302E32302E3271177562580C0000006E5F657374696D61746F727371184B025810000000657374696D61746F725F706172616D737119286807680B680C680D680E680F681168126813681074711A5809000000626F6F747374726170711B8858090000006F6F625F73636F7265711C8958060000006E5F6A6F6273711D4E68104E5807000000766572626F7365711E4B00580A0000007761726D5F7374617274711F8968144E68076808680B4B02680C4B02680D4B01680E470000000000000000680F58040000006175746F712068114E681247000000000000000068134E580B0000006E5F66656174757265735F71214B0D580A0000006E5F6F7574707574735F71224B015808000000636C61737365735F7123636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A7124636E756D70790A6E6461727261790A71254B008571264301627127877128527129284B014B0385712A636E756D70790A64747970650A712B58020000006934712C4B004B0187712D52712E284B0358010000003C712F4E4E4E4AFFFFFFFF4AFFFFFFFF4B007471306289430C000000000100000002000000713174713262580A0000006E5F636C61737365735F71334B03580F000000626173655F657374696D61746F725F71346805580B000000657374696D61746F72735F71355D7136286804298171377D713828680768086809680A680B4B02680C4B02680D4B01680E470000000000000000680F682068104A5BF54F2F68114E681247000000000000000068134E68144E68158968214B0D68224B016823682468254B00857139682787713A52713B284B014B0385713C682B58020000006638713D4B004B0187713E52713F284B03682F4E4E4E4AFFFFFFFF4AFFFFFFFF4B00747140628943180000000000000000000000000000F03F00000000000000407141747142626833636E756D70792E636F72652E6D756C746961727261790A7363616C61720A7143682B5802000000693871444B004B01877145527146284B03682F4E4E4E4AFFFFFFFF4AFFFFFFFF4B007471476243080300000000000000714886714952714A580D0000006D61785F66656174757265735F714B4B035805000000747265655F714C63736B6C6561726E2E747265652E5F747265650A547265650A714D4B0D682468254B0085714E682787714F527150284B014B01857151684689430803000000000000007152747153624B018771545271557D715628680B4B02580A0000006E6F64655F636F756E7471574B0758050000006E6F6465737158682468254B00857159682787715A52715B284B014B0785715C682B5803000000563536715D4B004B0187715E52715F284B0358010000007C71604E28580A0000006C6566745F6368696C647161580B00000072696768745F6368696C647162580700000066656174757265716358090000007468726573686F6C6471645808000000696D7075726974797165580E0000006E5F6E6F64655F73616D706C65737166581700000077656967687465645F6E5F6E6F64655F73616D706C657371677471687D716928686168464B0086716A686268464B0886716B686368464B1086716C6864683F4B1886716D6865683F4B2086716E686668464B2886716F6867683F4B30867170754B384B014B1074717162894288010000010000000000000004000000000000000B00000000000000000000D0A3700040A05CAEBD43D0E43F780000000000000000000000004066400200000000000000030000000000000003000000000000000000000000803140A83AB05E9BBBC43F1F000000000000000000000000804640FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFF00000000000000C0000000000000000003000000000000000000000000001040FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFFFFFFFFFF00000000000000C000000000000000001C000000000000000000000000804440050000000000000006000000000000000C0000000000000000000000001887408283A9172585E03F59000000000000000000000000A060

**5. Predykcja**

In [98]:
CREATE OR ALTER PROCEDURE predict_wine (@model VARCHAR(100))
AS
BEGIN
    DECLARE @nb_model VARBINARY(max) = (
            SELECT model
            FROM wine_models
            WHERE model_name = @model
            );

    EXECUTE sp_execute_external_script @language = N'Python'
        , @script = N'
import pickle
import pandas as pd

wine_model = pickle.loads(nb_model)

target = wine_data["WineId"]
wine_data.drop(labels=["WineId", "id"], axis=1, inplace=True)

OutputDataSet = pd.DataFrame()
OutputDataSet["WineId"] = target
OutputDataSet["Predicted"] = wine_model.predict(wine_data)
'
        , @input_data_1 = N'SELECT TOP 20 * FROM wine_data ORDER BY NEWID()'
        , @input_data_1_name = N'wine_data'
        , @params = N'@nb_model varbinary(max)'
        , @nb_model = @nb_model
    WITH RESULT SETS((
        "WineId" INT,
        "WineId.Predicted" INT
        ));
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.009

In [101]:
EXECUTE predict_wine 'RandomForest';
GO

(20 rows affected)

Total execution time: 00:00:04.295

WineId,WineId.Predicted
0,0
1,1
2,1
1,1
0,0
2,2
1,1
2,2
2,2
2,2


# SQL in Jupyter Notebooks
### Prerequisites
**Installing Python packages**
```
pip install ipython-sql
pip install pyodbc
```
[ipython-sql documentation](https://pypi.org/project/ipython-sql/)

**SQL driver installation**
[ODBC Driver 18 for SQL Server](https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16)
(Tested on MS SQL Server 2022)

### Connection to SQL Server
**%sql** for single-line queries.
**%%sql** for multi-line queries.
The **<<** operator captures query results in a local variable.

In [ ]:
# Loading magic method sql
%load_ext sql

# Connection
user = 'SA'
password = 'dockersql#1' #'YourPassword'
server = 'localhost'
port = '1433'
database = 'master'
driver = 'odbc+driver+18+for+sql+server'

connection_string = f'mssql+pyodbc://{user}:{password}@{server}:{port}/{database}?encrypt=no&driver={driver}'

%sql $connection_string

In [ ]:
# Test connection
%sql SELECT * FROM sys.databases